## 변형 오토인코더
변형 오토인코더(Variational autoencoder)를 좀 더 쉽게 이해할 수 있게 오토인코더와 비교하면서 설명하겠습니다. 오토인코더는 다음 그림과 같이  
`입력(숫자 2)` -> `인코더` -> `압축(차원축소)` -> `디코더` -> `출력(숫자2)`  
이 나오게 하는 방법입니다.  
![](../Static/695_2.jpg)  

오토인코더는 차원을 줄이는 것이 목표이기 때문에 새롭게 생성된 데이터의 확률 분포에는 관심이 없습니다.

반면 변형 오토인코더는 표준편차와 평균을 이용하여 확률 분포를 만들고, 거기에서 샘플링하여 디코더를 통과시킨 후 새로운 데이터를 만들어 냅니다. 즉, 변형 오토인코더는 `입력 데이터와 조금 다른 출력 데이터`를 만들어 내는데, 이때 z라는 가우시안 분포를 이용합니다.(z를 잠재벡터(latent vector)라고 합니다.) 중요한 특성의 파라미터를 담고 있는 z 분포에서 벡터를 랜덤하게 샘플링하고 이 분포의 오차를 이용하여 입력 데이터와 유사한 다양한 데이터를 만들어 내는 것이 변형 오토인코더입니다.  

![](../Static/696_1.jpg)

오토인코더는 데이터 벡터에 대한 차원을 축소하여 실제 이미지와 동일한 이미지를 출력하는 것이 목적이었다면, 변형 오토인코더는 데이터가 만들어지는 확률 분포를 찾아 비슷한 데이터를 생성하는 것이 목적입니다.

변형 오토인코더에서 인코더와 디코더에 대한 네트워크는 다음 그림과 같습니다.  
![](../Static/696_2.jpg)  

* qϕ(z|x) : x를 입력받아 잠재 벡터 z와 대응되는 평균과 분산을 구하는 네트워크로 인코더 네트워크를 의미합니다.

* pθ(x|z) : z를 입력받아 x와 대응되는 평균과 분산을 구하는 네트워크로 디코더 네트워크를 의미합니다.

그럼 인코더 네트워크부터 자세히 살펴보겠습니다.  
![](../Static/697_1.jpg)

이번 예제에서는 텐서보드에서 에포크 진행에 따른 오차를 확인할 예정입니다. 따라서 다음 명령으로 텐서보드를 설치합니다. 텐서보드 사용을 위해 `텐서보드 엑스(tensorboardX)` 라이브러리를 설치해야 합니다.

pip install rensorboardX

텐서보드 엑스는 학습 과정을 시각적으로 확인하고자 할 때 사용하는 도구입니다.

변형 오토인코더에서도 오토인코더에서와 마찬가지로 MNIST 데이터셋을 이용합니다. 먼저 필요한 라이브러리를 호출합니다.

In [1]:
import datetime
import os
from tensorboardX import SummaryWriter

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

import torchvision.datasets as datasets
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MNIST 데이터셋을 내려받은 후 이미지를 텐서로 변환합니다.

In [2]:
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = datasets.MNIST(
    root='./MNIST_DATA/',
    train=True,
    transform=transform,
    download=True,
)

test_dataset = datasets.MNIST(
    root='./MNIST_DATA/',
    train=False,
    transform=transform,
    download=True,
)

train_loader = DataLoader(
    dataset=train_dataset, batch_size=100, shuffle=True, num_workers=4, pin_memory=False
)

test_loader = DataLoader(
    dataset=test_dataset, batch_size=100, shuffle=False, num_workers=4
)



모델의 네트워크를 생성합니다. 네트워크는 오토인코더처럼 인코더와 디코더로 구성됩니다.

In [3]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()
        self.input1 = nn.Linear(input_dim, hidden_dim)
        self.input2 = nn.Linear(input_dim, hidden_dim)
        self.mean = nn.Linear(hidden_dim, latent_dim)
        self.var = nn.Linear(hidden_dim, latent_dim)

        self.LeakyReLU = nn.LeakyReLU(0.2)
        self.training = True
    
    def forward(self, x):
        h_ = self.LeakyReLU(self.input1(x))
        h_ = self.LeakyReLU(self.input2(h_))
        mean = self.mean(h_)
        log_var = self.var(h_)
        return mean, log_var # 인코더 네트워크에서 평균과 분산을 반환



인코더 역할은 데이터(x)가 주어졌을 때 디코더가 원래 데이터로 잘 복원할 수 있는 이상적인 확률 분포 p(z|x)를 찾는 것입니다. 변형 오토인코더에서는 이상적인 확률 분포를 찾는데 변분추론을 사용합니다.

이번에는 디코더 네트워크를 정의합니다.

In [4]:
class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.hidden1 = nn.Linear(latent_dim, hidden_dim)
        self.hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)
        self.LeakyReLU = nn.LeakyReLU(0.2)

    def forward(self, x):
        h = self.LeakyReLU(self.hidden1(x))
        h = self.LeakyReLU(self.hidden2(h))
        x_hat = torch.sigmoid(self.output(h))
        return x_hat # 디코더 결과는 시그모이드를 통과했으므로, 0~1 값을 갖습니다.

디코더는 추출한 샘플을 입력으로 받아 다시 원본으로 재구축(재생성)하는 역할을 합니다.